## Initial configuration

### Importing required packages

In [ ]:
from pathlib import Path

import requests
from tqdm import tqdm

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import altair as alt
import warnings

### Configuring packages

In [ ]:
sns.set_style('white')

alt.themes.enable('default')
alt.renderers.enable('kaggle')
alt.renderers.set_embed_options(actions=False)
alt.data_transformers.enable('json')
warnings.filterwarnings("ignore", 'This pattern has match groups')

## Data Loading

### Setting dataset location 

In [ ]:
DATA_PATH = Path('../input/airbnb/')

### Loading data

In [ ]:
listings_df = pd.read_csv(DATA_PATH/'listings_summary.csv',parse_dates = ['last_review'])
listings_detail_df = pd.read_csv(DATA_PATH/'listings.csv', low_memory = False, parse_dates = ['host_since', 'last_scraped', 'calendar_last_scraped', 'first_review', 'last_review'])

reviews_df = pd.read_csv(DATA_PATH/'reviews_summary.csv', parse_dates=['date'])
reviews_detail_df = pd.read_csv(DATA_PATH/'reviews.csv', parse_dates=['date'])

calendar_df = pd.read_csv(DATA_PATH/'calendar.csv', parse_dates=['date'])

neighborhood_df = pd.read_csv(DATA_PATH/'neighbourhoods.csv')
gdf = gpd.read_file(DATA_PATH/'neighbourhoods.geojson')

## Data Exploration

### listings_summary

#### Data overview

In [ ]:
print(f'Number of rows: {listings_df.shape[0]}\nNumber of cols: {listings_df.shape[1]}')
listings_df.sample(3)

#### Displaying number of nullable records

In [ ]:
listings_df.loc[:, listings_df.isnull().sum() > 0].isnull().sum().sort_values(ascending=False)

#### Displaying number of records without variance (having same value)

In [ ]:
listings_df.loc[:, listings_df.nunique() <= 1].nunique().sort_values()

#### Plotting neighbourhood values into chart

In [ ]:
listings_df['neighbourhood'].value_counts().sort_values().plot.barh(figsize=(10, 10));
plt.title('Number of listings by neighbourhood');

#### Plotting room_type values into chart

In [ ]:
listings_df['room_type'].value_counts(dropna=False).sort_values().plot.barh()
plt.title('Number of listings by room type')

### listings

#### Data overview

In [ ]:
print(f'Number of rows: {listings_detail_df.shape[0]}\nNumber of cols: {listings_detail_df.shape[1]}')
listings_detail_df.sample(3)

#### Displaying number of nullable records

In [ ]:
listings_detail_df.loc[:, listings_detail_df.isnull().sum() > 0].isnull().sum().sort_values(ascending=False)

#### Displaying number of records without variance (having same value)

In [ ]:
listings_detail_df.loc[:, listings_detail_df.nunique() <= 1].nunique().sort_values()

#### Couting number of rows with missing review score (there are 7 columns containing reviews score for different scopes)

In [ ]:
listings_detail_df.filter(regex='review_scores').isnull().sum(axis=1).value_counts()

#### Checking country_code to make sure it's from GB

In [ ]:
print(listings_detail_df['country_code'].value_counts())
listings_detail_df.query('country_code != "GB"')

### calendar

#### Data overview

In [ ]:
print(f'Number of rows: {calendar_df.shape[0]}\nNumber of cols: {calendar_df.shape[1]}')
calendar_df.sample(3)

#### Displaying number of nullable records

In [ ]:
calendar_df.loc[:, calendar_df.isnull().sum() > 0].isnull().sum().sort_values(ascending=False)

### reviews

#### Data overview

In [ ]:
print(f'Number of rows: {reviews_df.shape[0]}\nNumber of cols: {reviews_df.shape[1]}')
reviews_df.sample(3)

#### Displaying number of nullable records

In [ ]:
reviews_df.isnull().sum()

### reviews_summary

#### Data oveview

In [ ]:
print(f'Number of rows: {reviews_detail_df.shape[0]}\nNumber of cols: {reviews_detail_df.shape[1]}')
reviews_detail_df.sample(3)

#### Displaying number of nullable records

In [ ]:
reviews_detail_df.loc[:, reviews_detail_df.isnull().sum() > 0].isnull().sum().sort_values(ascending=False)

### neighbourhoods

#### Data overview

In [ ]:
print(f'Number of rows: {neighborhood_df.shape[0]}\nNumber of cols: {neighborhood_df.shape[1]}')
neighborhood_df.sample(3)

#### Displaying number of nullable records

In [ ]:
neighborhood_df.loc[:, neighborhood_df.isnull().sum() > 0].isnull().sum().sort_values(ascending=False)

### neighbourhoods.geojson

#### Data overview

In [ ]:
print(f'Number of rows: {gdf.shape[0]}\nNumber of cols: {gdf.shape[1]}')
gdf.sample(3)

#### Displaying number of nullable records

In [ ]:
gdf.loc[:, gdf.isnull().sum() > 0].isnull().sum().sort_values(ascending=False)

## Data Cleaning

### Dropping columns containing not useful data and containing texts (e.g. description) because NLP won't be used

In [ ]:
cols_to_drop = ['listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications', 'calendar_last_scraped', 'calendar_updated', 'availability_30', 'availability_60', 'availability_365', 'number_of_reviews_ltm', 'reviews_per_month']
df = listings_detail_df.drop(cols_to_drop, axis=1)

### Dropping columns containing mostly null values 

In [ ]:
df.drop(['host_acceptance_rate', 'neighbourhood_group_cleansed', 'square_feet', 'weekly_price', 'monthly_price', 'license', 'jurisdiction_names'], axis=1, inplace=True)

df.set_index('id', inplace=True)

### Dropping multiple columns that describe host listings count, leaving only host_listings_count

In [ ]:
df.drop(['host_total_listings_count', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], axis=1, inplace=True)

### Saving latitude and longitude to later compare if it's better to use districts or coordinates

In [ ]:
coordinates = df[['latitude', 'longitude']]

### Dropping columns describing location, leaving only neighbourhood_cleansed that contains district name

In [ ]:
df.drop(['zipcode', 'latitude', 'longitude', 'street', 'neighbourhood', 'city', 'state', 'market', 'smart_location', 'country_code', 'country', 'is_location_exact'], axis=1, inplace=True)

### Dropping columns describing max/min/avr of minimum/maximum number of nights guest has to spend, leaving only minimum_nights and maximum_nigths

In [ ]:
df.drop(['minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'], axis=1, inplace=True)

### Replacing string values to corresponding boolean values

In [ ]:
binary_map = {'f': False, 't': True}
df.replace(binary_map, inplace=True)

### Displaying data as histograms to better visualize lack of variance in some columns

In [ ]:
binary_map = {False: 0, True: 1}
df.replace(binary_map).hist(figsize=(20,20))
plt.show()

### Dropping columns without variance

In [ ]:
df.drop(['has_availability', 'host_has_profile_pic', 'is_business_travel_ready', 'require_guest_phone_verification', 'require_guest_profile_picture', 'requires_license'], axis=1, inplace=True)

### Displaying values in experiences_offered

In [ ]:
df.experiences_offered.value_counts(normalize=True)

### Dropping experiences_offered column with mostly none value

In [ ]:
df.drop('experiences_offered', axis=1, inplace=True)

### Checking which columns need to be cleaned

In [ ]:
df.loc[:, df.isnull().sum() > 0].isnull().sum().sort_values(ascending=False)

### security_deposit

#### Looking at data

In [ ]:
df['security_deposit'].sample(3)

#### Replacing null values

In [ ]:
df.security_deposit.fillna('$0.00', inplace=True)

#### Casting values to integer

In [ ]:
df.security_deposit = df.security_deposit.str[1:-3]
df.security_deposit = df.security_deposit.str.replace(",", "")
df.security_deposit = df.security_deposit.astype('int64')

### host_response_time

#### Looking at data

In [ ]:
df['host_response_time'].sample(3)

In [ ]:
df.groupby('host_response_time').sum()

#### Replacing null values

In [ ]:
df.host_response_time.fillna("unknown", inplace=True)
df.host_response_time.value_counts(normalize=True)

### host_response_rate

#### Looking at data

In [ ]:
df['host_response_rate'].sample(3)

#### Replacing null values

In [ ]:
df.host_response_rate.fillna('unknown', inplace=True)
df.host_response_rate.value_counts(normalize=True)

### price

#### Looking at data

In [ ]:
df['price'].sample(3)

#### Casting values to integer

In [ ]:
df.price = df.price.str[1:-3]
df.price = df.price.str.replace(",", "")
df.price = df.price.astype('int64')

### cleaning_fee

#### Looking at data

In [ ]:
df['cleaning_fee'].sample(3)

#### Replacing null values

In [ ]:
df.cleaning_fee.fillna('$0.00', inplace=True)

#### Casting values to integer

In [ ]:
df.cleaning_fee = df.cleaning_fee.str[1:-3]
df.cleaning_fee = df.cleaning_fee.str.replace(",", "")
df.cleaning_fee = df.cleaning_fee.astype('int64')

### extra_people

#### Looking at data

In [ ]:
df['extra_people'].sample(3)

#### Replacing null values

In [ ]:
df.extra_people.fillna('$0.00', inplace=True)

#### Casting values to integer

In [ ]:
df.extra_people = df.extra_people.str[1:-3]
df.extra_people = df.extra_people.str.replace(",", "")
df.extra_people = df.extra_people.astype('int64')

### host_since

#### Looking at data

In [ ]:
#can be replaced
print(len(df[df.loc[ :,['host_since'] ].isnull().sum(axis=1) == 1]))
df[df.loc[ :,['host_since'] ].isnull()].sample(3)

#### Removing null values

In [ ]:
df.dropna(subset=['host_since'], inplace=True)

### property_type

#### Looking at data

In [ ]:
df.property_type.value_counts()

#### Unifying property types

In [ ]:
property_map = {
    'Townhouse': 'House',
    'Serviced apartment': 'Apartment',
    'Loft': 'Apartment',
    'Bungalow': 'House',
    'Cottage': 'House',
    'Villa': 'House',
    'Tiny house': 'House',
    'Earth house': 'House',
    'Chalet': 'House'
}
df.property_type.replace(property_map, inplace=True)

#### Unifying other types into one group

In [ ]:
df.loc[~df.property_type.isin(['House', 'Apartment']), 'property_type'] = 'Other'

### bed_type

#### Looking at data

In [ ]:
df.bed_type.value_counts(normalize = True)

#### Dropping column without variance

In [ ]:
df.drop('bed_type', axis=1, inplace=True)

### bathrooms

#### Looking at data

In [ ]:
df.bathrooms.value_counts(normalize = True)

#### Dropping null values

In [ ]:
df.dropna(subset=['bathrooms'], inplace=True)

#### Casting values to integer

In [ ]:
df.bathrooms = df.bathrooms.astype('int64')

### bedrooms

#### Looking at data

In [ ]:
df.bedrooms.value_counts(normalize = True)

#### Dropping null values

In [ ]:
df.dropna(subset=['bedrooms'], inplace=True)

#### Casting values to integer

In [ ]:
df.bedrooms = df.bedrooms.astype('int64')

### beds

#### Looking at data

In [ ]:
df.beds.value_counts(normalize = True)

#### Dropping null values

In [ ]:
df.dropna(subset=['beds'], inplace=True)

#### Casting values to integer

In [ ]:
df.beds = df.beds.astype('int64')

### first_review

#### Looking at data

In [ ]:
df.first_review.value_counts(normalize = True)

#### Converting values to datetime

In [ ]:
df.first_review = pd.to_datetime(df.first_review)

#### Replacing null values

In [ ]:
#df['first_review'].fillna('no reviews', inplace=True)

### last_review

#### Looking at data

In [ ]:
df.last_review.value_counts(normalize = True)

#### Converting values to datetime

In [ ]:
df.last_review = pd.to_datetime(df.last_review)

#### Replacing null values

In [ ]:
#df['last_review'].fillna('no reviews', inplace=True)

### cancellation_policy

#### Looking at data

In [ ]:
df.cancellation_policy.value_counts(normalize = True)

#### Grouping policies into 3 main avalible on airbnb right now

In [ ]:
cancellation_policy_map = {
    'super_strict_30': 'strict_14_with_grace_period',
    'super_strict_60': 'strict_14_with_grace_period',
    'strict': 'strict_14_with_grace_period',
    'luxury_moderate': 'moderate'
}
df.cancellation_policy.replace(cancellation_policy_map, inplace=True)

### review_scores

#### Replacing null values

In [ ]:
review_cols = ['review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']

#for col in review_cols:
    #df[col].fillna(0, inplace=True)
    #df[col] = df[col].astype('int64')

### amenities

#### Creating columns for the most important amenities

In [ ]:
df.loc[df['amenities'].str.contains('Self check-in'), 'self_check_in'] = 1
df.loc[df['amenities'].str.contains('24-hour check-in'), 'check_in_24h'] = 1
df.loc[df['amenities'].str.contains('Long term stays allowed'), 'long_term_stays'] = 1

df.loc[df['amenities'].str.contains('Breakfast'), 'breakfast'] = 1
df.loc[df['amenities'].str.contains('Coffee maker|Espresso machine'), 'coffee_machine'] = 1
df.loc[df['amenities'].str.contains('Cooking basics|Dishwasher|Dryer|Washer'), 'appliances'] = 1

df.loc[df['amenities'].str.contains('Garden|Outdoor|Sun loungers|Terrace'), 'outdoor_space'] = 1
df.loc[df['amenities'].str.contains('Balcony|Patio'), 'balcony'] = 1
df.loc[df['amenities'].str.contains('Bed linens'), 'bed_linen'] = 1

df.loc[df['amenities'].str.contains('TV'), 'tv'] = 1
df.loc[df['amenities'].str.contains('Internet|Pocket wifi|Wifi'), 'internet'] = 1

df.loc[df['amenities'].str.contains('Family/kid friendly|Children|children'), 'family_friendly'] = 1
df.loc[df['amenities'].str.contains('Pets|pet|Cat(s)|Dog(s)'), 'pets_allowed'] = 1

df.loc[df['amenities'].str.contains('Private entrance'), 'private_entrance'] = 1
df.loc[df['amenities'].str.contains('Smoking allowed'), 'smoking_allowed'] = 1
df.loc[df['amenities'].str.contains('Elevator|Step-free access|Wheelchair|Accessible'), 'accessible'] = 1

#### Dropping column

In [ ]:
df.drop('amenities', axis=1, inplace=True)

## Data Visualization

In [ ]:
plt.figure(figsize=(20,4))
df.price.hist(bins=100, range=(0,1000))
plt.margins(x=0)
plt.axvline(200, color='green')
plt.axvline(500, color='orange')
plt.axvline(1000, color='red')
plt.title("Airbnb advertised nightly prices in London up to £1000", fontsize=16)
plt.xlabel("Price (£)")
plt.ylabel("Number of listings")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
df.groupby('accommodates').price.median().plot(kind='bar')
plt.title('Median price of Airbnbs accommodating different number of guests', fontsize=14)
plt.xlabel('Number of guests accommodated', fontsize=13)
plt.ylabel('Median price (£)', fontsize=13)
plt.xticks(rotation=0)
plt.show()

In [ ]:
df[['accommodates', 'bathrooms', 'bedrooms', 'beds']].hist(figsize=(8,6));

In [ ]:
# Dropping the empty column
neighborhood_df.drop('neighbourhood_group', axis=1, inplace=True)

In [ ]:
gdf.drop('neighbourhood_group', axis=1, inplace=True)

In [ ]:
gdf

In [ ]:
gdf.rename(columns={'neighbourhood' : 'neighborhood'}, inplace=True)
gdf = gdf.set_index('neighborhood')

In [ ]:
def gen_map_chart(df, review_col, review_title):
    return alt.Chart(df, title=review_title).mark_geoshape().encode(color=f'{review_col}:Q', tooltip=['neighborhood:N', f'{review_col}:Q']).properties(width=250, height=250)

In [ ]:
review_cols = ['review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']

listing_detail_cols = ['instant_bookable', 'neighbourhood_cleansed', 'room_type'] + review_cols

map_df = df[df.filter(regex='review_scores').notnull().all(axis=1)]
map_df.rename(columns={'neighbourhood_cleansed' : 'neighborhood'}, inplace=True)
map_df.set_index('neighborhood', inplace=True)

In [ ]:
geometry_df = gdf.loc[:, ['geometry']]
map_df = geometry_df.join(map_df)

In [ ]:
map_df['mean_review_scores_accuracy'] = map_df.groupby('neighborhood')['review_scores_accuracy'].transform('mean')
map_df['mean_review_scores_cleanliness'] = map_df.groupby('neighborhood')['review_scores_cleanliness'].transform('mean')
map_df['mean_review_scores_checkin'] = map_df.groupby('neighborhood')['review_scores_checkin'].transform('mean')
map_df['mean_review_scores_communication'] = map_df.groupby('neighborhood')['review_scores_communication'].transform('mean')
map_df['mean_review_scores_location'] = map_df.groupby('neighborhood')['review_scores_location'].transform('mean')
map_df['mean_review_scores_value'] = map_df.groupby('neighborhood')['review_scores_value'].transform('mean')

map_df['mean_review_scores_all'] = map_df.filter(like='mean_review_scores').mean(axis=1)

In [ ]:
map_df['number_of_listings'] = map_df.groupby('neighborhood').size()
median_price_by_neighborhood = map_df.groupby('neighborhood').price.median().values

In [ ]:
map_df.reset_index(level=0, inplace=True)

In [ ]:
review_cols = ['mean_review_scores_accuracy', 'mean_review_scores_cleanliness', 'mean_review_scores_checkin', 'mean_review_scores_communication', 'mean_review_scores_location', 'mean_review_scores_value']
review_titles = ['Accuracy', 'Cleanliness', 'Check-in', 'Communication', 'Location', 'Value']
review_map = {col: title for col, title in zip(review_cols, review_titles)}

result_df = map_df[['geometry', 'neighborhood', 'mean_review_scores_all', 'number_of_listings'] + review_cols].drop_duplicates()

In [ ]:
result_df['median_price'] = median_price_by_neighborhood

In [ ]:
review_charts = []

for review_col, review_title in zip(review_cols, review_titles):
    review_charts.append(gen_map_chart(result_df, review_col, review_title))

overall_map_chart = gen_map_chart(result_df, 'mean_review_scores_all', 'Overall')

((alt.vconcat(alt.concat(*review_charts, columns=3), overall_map_chart, 
              title='Average review scores by neighbourhood', 
              center=True)
     .configure_view(strokeWidth=0)
     .configure_title(fontSize=18)
     .configure_legend(title=None, orient='top',  labelFontSize=12)))

In [ ]:
overall_charts = []

number_of_listings_map_chart = gen_map_chart(result_df, 'number_of_listings', 'Number of listings')

((number_of_listings_map_chart
     .configure_view(strokeWidth=0)
     .configure_title(fontSize=18)
     .configure_legend(title=None, orient='top',  labelFontSize=12)))

In [ ]:
overall_charts = []

price_map_chart = gen_map_chart(result_df, 'median_price', 'Price median')

((price_map_chart
     .configure_view(strokeWidth=0)
     .configure_title(fontSize=18)
     .configure_legend(title=None, orient='top',  labelFontSize=12)))

In [ ]:
result_df = (geo_listings_df[review_cols].rename(review_map, axis=1)
                                         .corr()
                                         .reset_index()
                                         .melt(id_vars='index')
                                         .rename({'value': 'correlation'}, axis=1))

base = alt.Chart(
    result_df,
    title='Average Review Scores Relationship'
).properties(
    width=600, 
    height=600
)

heatmap = base.mark_rect().encode(
    x=alt.X('index:N', title=None),
    y=alt.Y('variable:N', title=None),
    color='correlation:Q'
)

text = base.mark_text(baseline='middle').encode(
    x=alt.X('index:N', title=None),
    y=alt.Y('variable:N', title=None),
    text=alt.Text('correlation:Q', format='.2f'),
    color=alt.condition(
        alt.datum.correlation < 0,
        alt.value('black'),
        alt.value('white')
    )
)
(heatmap + text).configure_axis(
    labelAngle=0,
    labelFontSize=14
).configure_legend(
    orient='top',
    titleFontSize=14,    
).configure_title(
    fontSize=18,
    offset=15,
    anchor='start',
    frame='group'
)

In [ ]:
def gen_parallel_chart(df, class_col, class_title):
    result_df = (df.groupby(class_col)[review_cols]
                   .mean()
                   .reset_index()
                   .melt(id_vars=class_col))
    result_df['variable'] = result_df['variable'].map(review_map)
    chart = alt.Chart(
        result_df,
        title = f'{class_title}'
    ).mark_line().encode(
        x=alt.X('variable:N',
                title=None),
        y=alt.Y('value:Q',
                scale=alt.Scale(zero=False),
                axis=None),
        color=f'{class_col}:N'
    ).properties(
        width=750, 
        height=300
    )
    
    return chart

In [ ]:
class_cols = ['room_type', 'instant_bookable', 'host_is_superhost']
class_titles = ['Room Type', 'Listing is Instant Bookable', 'Host is Superhost']

charts = []

for class_col, class_title in zip(class_cols, class_titles):
    charts.append(gen_parallel_chart(geo_listings_df, class_col, class_title))

In [ ]:
(alt.concat(*charts, columns=1, title='Average Review Scores by Host/Listing Properties')
    .configure_view(strokeWidth=0)
    .configure_legend(
        title=None, 
        orient='top', 
        columns=0,
        labelFontSize=14)
    .configure_axis(
        labelAngle=0,
        grid=False,
        labelFontSize=14)
    .configure_title(
        anchor='start',
        fontSize=18,
        offset=15)
    .resolve_scale(color='independent')
)